# 📚 Classification Fundamentals and Problem Formulation (5 minutes)

### What is Supervised Classification?

**Supervised Classification** is a machine learning task where we train algorithms to predict discrete class labels for new data based on labeled training examples.

### Key Components:

1. **Training Data**: Labeled examples (X, y) where X = features, y = class labels
2. **Features**: Input variables that describe each sample
3. **Labels**: Target classes we want to predict
4. **Model**: Algorithm that learns the mapping from features to labels
5. **Evaluation**: Metrics to measure how well our model performs

### Our Classification Problem: 3W Oil Well Fault Detection

- **Objective**: Classify oil well operational states from sensor data
- **Input Features**: Time series sensor measurements (flattened into feature vectors)
- **Output Classes**: Different types of operational faults (0=normal, 1-9=different fault types)
- **Challenge**: Multi-class classification with imbalanced classes

### Why Classification Matters in Oil Wells:
- **Early Fault Detection**: Prevent costly equipment failures
- **Operational Safety**: Avoid dangerous situations
- **Maintenance Planning**: Schedule repairs before critical failures
- **Cost Reduction**: Minimize downtime and repair costs

### Problem Characteristics:
- **Multi-class**: 10 different classes (0-9)
- **Time Series**: Sequential sensor measurements
- **High Dimensional**: Many sensors × time steps = many features
- **Imbalanced**: Some fault types are much rarer than others
- **Real-world**: Noisy, complex industrial data

Let's explore different algorithms to solve this classification challenge!

In [1]:
# ============================================================
# LOAD 3W DATASET FOR SUPERVISED CLASSIFICATION
# ============================================================
import time

start_time = time.time()

print("🤖 Loading 3W Dataset for Supervised Classification")
print("=" * 55)

# Import data loading utilities
import sys
import os

sys.path.append("src")

print("📦 Importing modules...", end=" ")
from src.data_persistence import DataPersistence
from src import config
import pandas as pd
import numpy as np

print("✅")

try:
    print("📂 Initializing data persistence...", end=" ")
    persistence = DataPersistence(base_dir=config.PROCESSED_DATA_DIR, verbose=False)
    print("✅")

    print(f"⚡ Using format: {config.SAVE_FORMAT} for maximum speed")

    # Check if windowed directory exists
    windowed_dir = os.path.join(persistence.cv_splits_dir, "windowed")
    print(f"📁 Checking windowed directory: {windowed_dir}...", end=" ")

    if not os.path.exists(windowed_dir):
        print("❌")
        print(
            "❌ No windowed data directory found. Please run Data Treatment notebook first to generate windowed time series data."
        )
    else:
        print("✅")

        # Look for fold directories
        print("🔍 Looking for fold directories...", end=" ")
        fold_dirs = [
            d
            for d in os.listdir(windowed_dir)
            if d.startswith("fold_") and os.path.isdir(os.path.join(windowed_dir, d))
        ]
        fold_dirs.sort()
        print(f"✅ Found {len(fold_dirs)} folds")

        if not fold_dirs:
            print("❌ No fold directories found in windowed data.")
        else:
            # Load data from ALL folds for comprehensive classification
            print(
                f"📊 Loading windowed data from ALL {len(fold_dirs)} folds for classification..."
            )

            all_train_windows = []
            all_train_classes = []
            all_train_fold_info = []  # Track which fold each training sample comes from
            all_test_windows = []
            all_test_classes = []
            all_test_fold_info = []  # Track which fold each test sample comes from

            load_start = time.time()

            for fold_name in fold_dirs:
                fold_path = os.path.join(windowed_dir, fold_name)
                fold_num = fold_name.replace("fold_", "")

                print(f"📁 Processing {fold_name}...", end=" ")

                # Load training data
                train_pickle = os.path.join(
                    fold_path, f"train_windowed.{config.SAVE_FORMAT}"
                )
                train_parquet = os.path.join(fold_path, "train_windowed.parquet")

                if os.path.exists(train_pickle):
                    fold_train_dfs, fold_train_classes = persistence._load_dataframes(
                        train_pickle, config.SAVE_FORMAT
                    )
                    all_train_windows.extend(fold_train_dfs)
                    all_train_classes.extend(fold_train_classes)
                    all_train_fold_info.extend(
                        [fold_name] * len(fold_train_dfs)
                    )  # Track fold info
                elif os.path.exists(train_parquet):
                    fold_train_dfs, fold_train_classes = persistence._load_from_parquet(
                        train_parquet
                    )
                    all_train_windows.extend(fold_train_dfs)
                    all_train_classes.extend(fold_train_classes)
                    all_train_fold_info.extend(
                        [fold_name] * len(fold_train_dfs)
                    )  # Track fold info

                # Load test data
                test_pickle = os.path.join(
                    fold_path, f"test_windowed.{config.SAVE_FORMAT}"
                )
                test_parquet = os.path.join(fold_path, "test_windowed.parquet")

                if os.path.exists(test_pickle):
                    fold_test_dfs, fold_test_classes = persistence._load_dataframes(
                        test_pickle, config.SAVE_FORMAT
                    )
                    all_test_windows.extend(fold_test_dfs)
                    all_test_classes.extend(fold_test_classes)
                    all_test_fold_info.extend(
                        [fold_name] * len(fold_test_dfs)
                    )  # Track fold info
                elif os.path.exists(test_parquet):
                    fold_test_dfs, fold_test_classes = persistence._load_from_parquet(
                        test_parquet
                    )
                    all_test_windows.extend(fold_test_dfs)
                    all_test_classes.extend(fold_test_classes)
                    all_test_fold_info.extend(
                        [fold_name] * len(fold_test_dfs)
                    )  # Track fold info

                print("✅")

            load_time = time.time() - load_start

            if all_train_windows and all_test_windows:
                print(f"✅ Successfully loaded windowed data from ALL folds!")
                print(f"🚂 Training windows: {len(all_train_windows)}")
                print(f"🧪 Test windows: {len(all_test_windows)}")
                print(f"⚡ Loading time: {load_time:.3f} seconds")

                # Store for further processing
                train_dfs = all_train_windows
                train_classes = all_train_classes
                train_fold_info = all_train_fold_info  # Store fold tracking info
                test_dfs = all_test_windows
                test_classes = all_test_classes
                test_fold_info = all_test_fold_info  # Store fold tracking info

                # Display sample window information
                if train_dfs:
                    print("📋 Processing first training window...", end=" ")
                    first_train_window = train_dfs[0]
                    first_train_class = train_classes[0]
                    print("✅")

                    print(f"\n🪟 Sample Training Window (Window #1):")
                    print(f"   • Shape: {first_train_window.shape}")
                    print(f"   • Class: {first_train_class}")
                    print(f"   • Features: {list(first_train_window.columns)}")

                    # Show class distribution
                    print(f"\n📊 Training Set Class Distribution:")
                    train_unique, train_counts = np.unique(
                        train_classes, return_counts=True
                    )
                    for cls, count in zip(train_unique, train_counts):
                        print(f"   • Class {cls}: {count} windows")

                    print(f"\n📊 Test Set Class Distribution:")
                    test_unique, test_counts = np.unique(
                        test_classes, return_counts=True
                    )
                    for cls, count in zip(test_unique, test_counts):
                        print(f"   • Class {cls}: {count} windows")

                    total_time = time.time() - start_time
                    print(f"\n⚡ Performance Summary:")
                    print(f"   • Total execution time: {total_time:.3f} seconds")
                    print(f"   • Data loading time: {load_time:.3f} seconds")
                    print(f"   • File format: {config.SAVE_FORMAT}")
                    print(f"   • Folds processed: {len(fold_dirs)}")

                    print(f"\n🎯 Dataset Summary for Supervised Classification:")
                    print(f"   • Total training windows: {len(train_dfs)}")
                    print(f"   • Total test windows: {len(test_dfs)}")
                    print(f"   • Window dimensions: {first_train_window.shape}")
                    print(f"   • Classes available: {sorted(train_unique)}")
                    print(f"   • Ready for: Decision Trees, SVM, Neural Networks")

                else:
                    print("⚠️ No training windows found in any fold")
                    train_dfs = []
                    train_classes = []
                    train_fold_info = []
                    test_dfs = []
                    test_classes = []
                    test_fold_info = []

except Exception as e:
    print(f"❌ Error loading data: {str(e)}")
    print(f"\n💡 Troubleshooting:")
    print(f"   1. Make sure 'Data Treatment.ipynb' ran completely")
    print(f"   2. Check if windowed data was saved successfully")
    print(f"   3. Verify the processed_data directory exists")
    print(f"   4. Ensure pickle format is available for fast loading")

    # Show directory status
    expected_dir = config.PROCESSED_DATA_DIR
    print(f"\n📁 Directory check: {expected_dir}")
    if os.path.exists(expected_dir):
        print(f"✅ Base directory exists")
        windowed_path = os.path.join(expected_dir, "cv_splits", "windowed")
        if os.path.exists(windowed_path):
            print(f"✅ Windowed directory exists")
            try:
                contents = os.listdir(windowed_path)
                print(f"📄 Contents: {contents}")
            except:
                print("❌ Cannot list directory contents")
        else:
            print(f"❌ Windowed directory missing: {windowed_path}")
    else:
        print(f"❌ Base directory does not exist")

    # Initialize empty variables for error case
    train_dfs = []
    train_classes = []
    train_fold_info = []
    test_dfs = []
    test_classes = []
    test_fold_info = []

🤖 Loading 3W Dataset for Supervised Classification
📦 Importing modules... ✅
📂 Initializing data persistence... ✅
⚡ Using format: pickle for maximum speed
📁 Checking windowed directory: processed_data\cv_splits\windowed... ✅
🔍 Looking for fold directories... ✅ Found 3 folds
📊 Loading windowed data from ALL 3 folds for classification...
📁 Processing fold_1... ✅
📁 Processing fold_2... ✅
📁 Processing fold_3... ✅
✅ Successfully loaded windowed data from ALL folds!
🚂 Training windows: 505336
🧪 Test windows: 78491
⚡ Loading time: 94.042 seconds
📋 Processing first training window... ✅

🪟 Sample Training Window (Window #1):
   • Shape: (300, 4)
   • Class: 0
   • Features: ['P-PDG_scaled', 'P-TPT_scaled', 'T-TPT_scaled', 'class']

📊 Training Set Class Distribution:
   • Class 0: 52936 windows
   • Class 1: 80335 windows
   • Class 2: 5796 windows
   • Class 3: 46257 windows
   • Class 4: 10236 windows
   • Class 5: 130401 windows
   • Class 6: 368 windows
   • Class 7: 64760 windows
   • Class 

# 🔍 Enhanced Analysis Summary

## Five Key Analysis Features

The enhanced classification provides five critical analysis capabilities:

### 1. 🎯 **Configurable Class Selection**
- **Purpose**: Choose which specific classes to include in your analysis
- **Configuration**: Controlled via `src/config.py` for easy management
- **Options**: 
  - Default: Exclude only class 0 (analyze all fault types 1-9)
  - Custom: Select specific classes of interest (e.g., [2,3,8] for particular faults)
  - Presets: Use pre-configured selections for common scenarios
- **Use Case**: Focus on specific fault types, reduce problem complexity, targeted analysis

### 2. 📊 **Accuracy Per Fold**
- **Purpose**: Understand model consistency across different data splits
- **Insight**: Identifies if models perform consistently or if some folds are particularly challenging
- **Use Case**: Helps detect dataset bias, temporal patterns, or fold-specific issues

### 3. 🏷️ **Accuracy Per Class** 
- **Purpose**: Understand model performance on each selected class
- **Insight**: Reveals which fault types are easier/harder to detect
- **Use Case**: Critical for industrial applications where missing certain fault types has higher cost

### 4. ⚠️ **Flexible Class Filtering**
- **Purpose**: Either exclude normal operation (class 0) or focus on specific fault types
- **Insight**: Create focused classification problems aligned with operational needs
- **Use Case**: 
  - Fault diagnosis systems (exclude normal operation)
  - Specific fault type analysis (select particular faults)
  - Simplified problems for testing and development

### 5. 🔄 **Configurable Test Data Balancing**
- **Purpose**: Ensure robust evaluation with sufficient samples per class
- **Configuration**: Enable/disable via config file settings
- **Insight**: Balances test data to have minimum samples per selected class for reliable metrics
- **Use Case**: Prevents evaluation bias due to class imbalance in test set

## Configuration Management

### Config File Location:
```
📁 src/config.py
└── CLASSIFICATION_CONFIG section
└── CLASSIFICATION_PRESETS section
```

### Available Presets:
```python
CLASSIFICATION_PRESETS = {
    'all_faults': None,                # All fault types (exclude class 0)
    'specific_faults': [2, 3, 8],     # User-defined specific faults
    'early_faults': [1, 2, 3, 4, 5],  # First 5 fault types
    'late_faults': [7, 8, 9],         # Last 3 fault types
    'odd_faults': [1, 3, 5, 7, 9],    # Odd-numbered fault types
    'even_faults': [2, 4, 6, 8],      # Even-numbered fault types
    'critical_faults': [3, 6, 8, 9],  # Critical operational faults
    'minor_faults': [1, 2, 4, 5, 7],  # Minor operational faults
    'binary_test': [3, 8],            # Simple binary classification
}
```

### Configuration Examples:

#### In config.py:
```python
CLASSIFICATION_CONFIG = {
    'selected_classes': [2, 3, 8],     # Default selection
    'balance_test': False,             # Test balancing setting
    'min_test_samples_per_class': 300, # Min samples when balancing
    'balance_classes': True,           # Training balancing
    'balance_strategy': 'combined',    # Balancing strategy
    'max_samples_per_class': 1000,     # Training limit
    'verbose': True                    # Progress display
}
```

#### In notebook override:
```python
# Quick override in notebook
selected_classes = config.CLASSIFICATION_PRESETS['binary_test']  # Use preset
# OR
selected_classes = [1, 4, 7]  # Custom selection
```

## Why Configuration Management Matters

### Easy Experimentation:
- **Quick Changes**: Modify config file without editing notebook code
- **Preset Library**: Use common configurations instantly
- **Version Control**: Track configuration changes easily
- **Reproducibility**: Share exact configurations with team

### Industrial Relevance:
- **Deployment Ready**: Production configurations separate from code
- **Operational Flexibility**: Adapt to different operational scenarios
- **Maintenance Efficiency**: Update configurations without code changes
- **Team Collaboration**: Shared configuration standards

### Model Development:
- **Systematic Testing**: Compare different class combinations systematically
- **Performance Optimization**: Focus computational resources efficiently
- **Iterative Development**: Progressive complexity increase
- **Configuration Documentation**: Track what works best for different scenarios

In [2]:
# ============================================================
# COMPREHENSIVE SUPERVISED CLASSIFICATION WITH ENHANCED ANALYSIS
# ============================================================

print("🤖 Running Enhanced Supervised Classification Analysis")
print("=" * 60)

# Check if we have loaded data from previous cell
if (
    "train_dfs" in locals()
    and train_dfs is not None
    and len(train_dfs) > 0
    and "test_dfs" in locals()
    and test_dfs is not None
    and len(test_dfs) > 0
):

    # Import the enhanced supervised classification module and config
    from src.supervised_classification import enhanced_fold_analysis
    from src import config

    print("📊 Using enhanced supervised classification module...")
    print(f"🚂 Training windows available: {len(train_dfs)}")
    print(f"🧪 Test windows available: {len(test_dfs)}")

    # Check if fold information is available
    fold_available = (
        "test_fold_info" in locals()
        and test_fold_info is not None
        and len(test_fold_info) == len(test_dfs)
    )

    if fold_available:
        unique_folds = sorted(set(test_fold_info))
        print(f"📁 Fold information available: {len(unique_folds)} folds detected")
        print(f"📁 Folds: {unique_folds}")
    else:
        print("⚠️ Fold information not available - will skip per-fold analysis")

    # ============================================================
    # LOAD CLASSIFICATION CONFIGURATION FROM CONFIG FILE
    # ============================================================

    print(f"\n⚙️ Loading classification configuration from config.py...")

    # Load configuration from config file
    classification_config = config.CLASSIFICATION_CONFIG
    classification_presets = config.CLASSIFICATION_PRESETS

    # You can easily change the configuration by modifying these lines:
    # Option 1: Use configuration directly from config file
    selected_classes = classification_config["selected_classes"]
    balance_test = classification_config["balance_test"]
    min_test_samples_per_class = classification_config["min_test_samples_per_class"]

    # Option 2: Override with a preset (uncomment one to use)
    # selected_classes = classification_presets['all_faults']         # All fault types
    # selected_classes = classification_presets['early_faults']       # Classes 1-5
    # selected_classes = classification_presets['late_faults']        # Classes 7-9
    # selected_classes = classification_presets['odd_faults']         # Odd classes
    # selected_classes = classification_presets['binary_test']        # Classes 3,8

    # Option 3: Custom selection (uncomment and modify)
    # selected_classes = [1, 4, 7]  # Your custom selection

    print(f"📋 Available classification presets:")
    for preset_name, preset_classes in classification_presets.items():
        print(f"   • {preset_name}: {preset_classes}")

    print(f"\n🎯 Current Classification Configuration:")
    print(f"   📊 Selected classes: {selected_classes}")
    print(f"   📊 Balance test data: {balance_test}")
    if balance_test:
        print(f"   📊 Min test samples per class: {min_test_samples_per_class}")

    if selected_classes is None:
        print(f"   📊 Analysis Mode: All fault types (exclude only class 0)")
        print(f"   📊 Classes to analyze: All available fault classes (1-9)")
    else:
        print(f"   📊 Analysis Mode: Custom class selection")
        print(f"   📊 Classes to analyze: {selected_classes}")

    # Run enhanced supervised classification with fold analysis
    # This will provide:
    # 1. Class filtering based on selected_classes parameter
    # 2. Accuracy per fold (if fold info available and test balancing disabled)
    # 3. Accuracy per class for selected classes only
    # 4. Confirmation of class selection status
    # Plus all the standard analysis (model comparison, visualizations, etc.)

    print("\n🔄 Starting enhanced classification pipeline...")
    print("📋 Analysis will include:")
    if selected_classes is None:
        print("   ⚠️ Complete exclusion of class 0 (normal operation)")
        print("   ✅ All fault types (classes 1-9) included")
    else:
        print(f"   🎯 Custom class selection: {selected_classes}")
        print("   ✅ Only selected classes included in analysis")
    print("   ✅ Standard model training (Decision Trees, SVM, Neural Networks)")
    print("   ✅ Data augmentation for class balancing (training)")
    if balance_test:
        print(
            f"   ✅ Test data balancing (ensures min {min_test_samples_per_class} samples per class)"
        )
    else:
        print("   ⚠️ Test data balancing disabled")
    print("   ✅ Per-class accuracy analysis (selected classes only)")
    if fold_available:
        print("   ⚠️ Per-fold analysis (may be disabled due to test balancing)")
    else:
        print("   ⚠️ Per-fold analysis (skipped - fold info unavailable)")

    classifier = enhanced_fold_analysis(
        train_dfs=train_dfs,  # Full training data
        train_classes=train_classes,  # Full training labels
        test_dfs=test_dfs,  # Full test data
        test_classes=test_classes,  # Full test labels
        fold_info=(
            test_fold_info if fold_available else None
        ),  # Pass fold info if available
        balance_classes=classification_config["balance_classes"],  # From config
        balance_strategy=classification_config["balance_strategy"],  # From config
        max_samples_per_class=classification_config[
            "max_samples_per_class"
        ],  # From config
        balance_test=balance_test,  # From config (configurable)
        min_test_samples_per_class=min_test_samples_per_class,  # From config (configurable)
        selected_classes=selected_classes,  # From config (configurable)
        verbose=classification_config["verbose"],  # From config
    )

    print(f"\n🎉 Enhanced Supervised Classification Complete!")
    print(f"✅ All models trained and compared")
    if selected_classes is None:
        print(f"⚠️ Class 0 (normal operation) completely excluded from analysis")
        print(f"✅ All fault types (classes 1-9) analyzed")
    else:
        print(f"🎯 Custom class selection applied: {selected_classes}")
        print(f"✅ Only selected classes analyzed")
    print(f"✅ Class balancing applied using data augmentation (training)")
    if balance_test:
        print(
            f"✅ Test data balanced to ensure robust evaluation (min {min_test_samples_per_class} per class)"
        )
    else:
        print(f"⚠️ Test data balancing disabled")
    print(f"✅ Per-class accuracy analysis completed (selected classes only)")
    if fold_available:
        print(f"⚠️ Per-fold accuracy analysis (may be disabled due to test balancing)")
    print(f"✅ Standard performance visualizations created")
    print(f"✅ Enhanced analysis provides comprehensive classification insights")

    print(f"\n💡 Configuration Tips:")
    print(f"   • Edit 'src/config.py' to change default settings")
    print(f"   • Use classification presets for common scenarios")
    print(f"   • Override settings in this cell for quick experiments")

    # Store classifier for further analysis if needed
    supervised_classifier = classifier

else:
    print(
        "❌ No data available. Please run the previous cell first to load training and test data."
    )
    supervised_classifier = None

🤖 Running Enhanced Supervised Classification Analysis
📊 Using enhanced supervised classification module...
🚂 Training windows available: 505336
🧪 Test windows available: 78491
📁 Fold information available: 3 folds detected
📁 Folds: ['fold_1', 'fold_2', 'fold_3']

⚙️ Loading classification configuration from config.py...
📋 Available classification presets:
   • all_faults: None
   • specific_faults: [2, 3, 8]
   • early_faults: [1, 2, 3, 4, 5]
   • late_faults: [7, 8, 9]
   • odd_faults: [1, 3, 5, 7, 9]
   • even_faults: [2, 4, 6, 8]
   • critical_faults: [3, 6, 8, 9]
   • minor_faults: [1, 2, 4, 5, 7]
   • binary_test: [3, 8]

🎯 Current Classification Configuration:
   📊 Selected classes: [1, 2, 3, 4, 5, 8, 9]
   📊 Balance test data: False
   📊 Analysis Mode: Custom class selection
   📊 Classes to analyze: [1, 2, 3, 4, 5, 8, 9]

🔄 Starting enhanced classification pipeline...
📋 Analysis will include:
   🎯 Custom class selection: [1, 2, 3, 4, 5, 8, 9]
   ✅ Only selected classes included 

IndexError: boolean index did not match indexed array along axis 0; size of axis is 31555 but size of corresponding boolean axis is 2664

## ️ Configuration-Based Classification (New!)

### Easy Configuration Management

The classification system now uses a configuration file approach for easy experimentation:

**📁 Configuration File:** `src/config.py`
- Contains all classification settings
- Pre-configured presets for common scenarios
- Easy to modify without changing notebook code

### Quick Configuration Examples:

#### 1. Use Default Configuration (Current: [2, 3, 8]):
```python
# Already configured in config.py
selected_classes = config.CLASSIFICATION_CONFIG['selected_classes']
```

#### 2. Use a Preset:
```python
selected_classes = config.CLASSIFICATION_PRESETS['binary_test']    # Classes [3, 8]
selected_classes = config.CLASSIFICATION_PRESETS['early_faults']   # Classes [1,2,3,4,5]
selected_classes = config.CLASSIFICATION_PRESETS['all_faults']     # All fault types (1-9)
```

#### 3. Custom Selection:
```python
selected_classes = [1, 4, 7]  # Your specific choice
```

### Test Data Balancing Control:
```python
balance_test = True   # Enable balanced test evaluation
balance_test = False  # Use original test distribution (faster)
```

### Benefits:
- **🚀 Quick Experiments**: Change focus without code modifications
- **📋 Reproducible**: Share exact configurations
- **⚙️ Production Ready**: Separate configuration from code
- **🔄 Version Control**: Track configuration changes

---

## 🌳 Decision Trees and Random Forest (8 minutes)

### Decision Trees
- **How it works**: Creates a tree-like model of decisions
- **Advantages**: Easy to interpret, handles both numerical and categorical data
- **Disadvantages**: Prone to overfitting, can be unstable

### Random Forest
- **How it works**: Combines many decision trees (ensemble method)
- **Advantages**: Reduces overfitting, more robust, provides feature importance
- **Disadvantages**: Less interpretable than single tree, can still overfit with noisy data

### Why Good for Oil Well Data:
- Handles high-dimensional features well
- Provides feature importance (which sensors are most important)
- Robust to outliers and noise
- Works well with time series features

In [ ]:
# ============================================================
# INDIVIDUAL ALGORITHM TRAINING (OPTIONAL)
# ============================================================

print("🌳 Individual Algorithm Training Example")
print("=" * 50)

# This cell demonstrates how to use individual algorithms from the module
# The comprehensive classification above already ran all algorithms

if "supervised_classifier" in locals() and supervised_classifier is not None:
    print("📊 Comprehensive classification already completed above!")
    print("🔍 Here's how to access individual algorithm results:")

    # Access results from the comprehensive run
    results = supervised_classifier.results

    # Find tree-based models
    tree_models = [
        r for r in results if "Tree" in r["model_name"] or "Forest" in r["model_name"]
    ]

    if tree_models:
        print(f"\n🌳 Tree-Based Models Performance:")
        print("-" * 40)
        for result in tree_models:
            print(f"{result['model_name']}:")
            print(f"   • Training Accuracy: {result['train_accuracy']:.3f}")
            print(f"   • Test Accuracy: {result['test_accuracy']:.3f}")
            print(f"   • Training Time: {result['training_time']:.3f}s")

            # Show feature importance for Random Forest
            if (
                "Random Forest" in result["model_name"]
                and "feature_importance" in result
            ):
                print(f"   • Top 5 Most Important Features:")
                feature_importance = result["feature_importance"]
                top_features_idx = np.argsort(feature_importance)[-5:][::-1]
                for i, idx in enumerate(top_features_idx, 1):
                    print(f"     {i}. Feature {idx}: {feature_importance[idx]:.4f}")
            print()

    print("💡 To train individual algorithms separately:")
    print(
        "   1. Use supervised_classifier.prepare_data() to get X_train, y_train, X_test, y_test"
    )
    print(
        "   2. Call supervised_classifier.train_decision_trees(X_train, y_train, X_test, y_test)"
    )
    print("   3. Or use supervised_classifier.train_svm() or train_neural_networks()")

    # Example of how to train just decision trees individually
    print(f"\n🔧 Example: Training only Decision Trees individually")
    print("(This would be useful if you only want specific algorithms)")

    # Note: Data is already prepared in the classifier
    print(
        "✅ Data preparation, class balancing, and augmentation already handled by module"
    )
    print("✅ All models already trained - see comprehensive results above")

else:
    print(
        "❌ No classifier available. Please run the comprehensive classification cell first."
    )
    print("💡 The comprehensive cell above handles:")
    print("   • Data preparation and class balancing with augmentation")
    print("   • Training of all algorithms (Decision Trees, SVM, Neural Networks)")
    print("   • Model comparison and visualization")
    print("   • Performance analysis and recommendations")

🌳 Individual Algorithm Training Example
❌ No classifier available. Please run the comprehensive classification cell first.
💡 The comprehensive cell above handles:
   • Data preparation and class balancing with augmentation
   • Training of all algorithms (Decision Trees, SVM, Neural Networks)
   • Model comparison and visualization
   • Performance analysis and recommendations


## ⚡ Support Vector Machines (4 minutes)

### How SVM Works:
- **Goal**: Find the optimal hyperplane that separates classes with maximum margin
- **Kernel Trick**: Map data to higher dimensions to make it linearly separable
- **Support Vectors**: Data points closest to the decision boundary

### SVM Advantages:
- Effective in high-dimensional spaces (perfect for our flattened time series)
- Memory efficient (only uses support vectors)
- Versatile (different kernels for different data patterns)

### SVM for Oil Well Data:
- Handles high-dimensional sensor data well
- RBF kernel can capture non-linear patterns in sensor readings
- Good for binary classification problems (normal vs fault)

In [ ]:
# ============================================================
# SVM RESULTS FROM COMPREHENSIVE CLASSIFICATION
# ============================================================

print("⚡ Support Vector Machines Results")
print("=" * 40)

if "supervised_classifier" in locals() and supervised_classifier is not None:
    print("📊 SVM models already trained in comprehensive classification!")

    # Access SVM results
    results = supervised_classifier.results
    svm_models = [r for r in results if "SVM" in r["model_name"]]

    if svm_models:
        print(f"\n⚡ SVM Performance Summary:")
        print("-" * 35)
        for result in svm_models:
            print(f"{result['model_name']}:")
            print(f"   • Training Accuracy: {result['train_accuracy']:.3f}")
            print(f"   • Test Accuracy: {result['test_accuracy']:.3f}")
            print(f"   • Training Time: {result['training_time']:.3f}s")
            print()

        # Show which SVM performed better
        best_svm = max(svm_models, key=lambda x: x["test_accuracy"])
        print(
            f"🏆 Best SVM: {best_svm['model_name']} (Test Accuracy: {best_svm['test_accuracy']:.3f})"
        )

        print(f"\n SVM Implementation Details:")
        print(f"   • Used subset sampling for computational efficiency")
        print(f"   • Linear SVM: Faster, good for linearly separable data")
        print(f"   • RBF SVM: Better for complex non-linear patterns")
        print(f"   • Data already normalized - optimal for SVM performance")
        print(f"   • Class balancing with augmentation improved SVM robustness")

    print(f"\n Module Benefits:")
    print(f"   ✅ Automatic subset sampling for large datasets")
    print(f"   ✅ Both Linear and RBF kernels tested")
    print(f"   ✅ Optimized hyperparameters")
    print(f"   ✅ Integrated with data augmentation for class balancing")

else:
    print("❌ No classifier results available.")
    print("💡 Run the comprehensive classification cell to see SVM results.")
    print("📋 The module automatically handles:")
    print("   • Subset sampling for SVM computational efficiency")
    print("   • Training both Linear and RBF SVM variants")
    print("   • Performance comparison with other algorithms")
    print("   • Integration with balanced dataset using augmentation")

## 🧠 Neural Networks and Deep Learning Basics (8 minutes)

### Neural Networks Fundamentals:
- **Neurons**: Basic processing units that compute weighted sums + activation
- **Layers**: Input layer → Hidden layers → Output layer
- **Backpropagation**: Learning algorithm that adjusts weights based on errors
- **Activation Functions**: Non-linear functions (ReLU, sigmoid, tanh)

### Why Neural Networks for Oil Well Data:
- **Automatic Feature Learning**: Can discover complex patterns in sensor data
- **Non-linear Relationships**: Capture complex interactions between sensors
- **Temporal Patterns**: Can model dependencies in time series data
- **Scalability**: Handle large amounts of high-dimensional data

### Types of Neural Networks:
1. **Multi-Layer Perceptron (MLP)**: Standard feedforward network
2. **Convolutional Neural Networks (CNN)**: Good for pattern recognition
3. **Recurrent Neural Networks (RNN/LSTM)**: Designed for sequential data

### Deep Learning Advantages:
- Learns features automatically from raw data
- Can model very complex relationships
- State-of-the-art performance on many tasks

In [ ]:
# ============================================================
# NEURAL NETWORKS RESULTS FROM COMPREHENSIVE CLASSIFICATION
# ============================================================

print("🧠 Neural Networks Results")
print("=" * 35)

if "supervised_classifier" in locals() and supervised_classifier is not None:
    print("📊 Neural Network models already trained in comprehensive classification!")

    # Access Neural Network results
    results = supervised_classifier.results
    nn_models = [r for r in results if "Neural Network" in r["model_name"]]

    if nn_models:
        print(f"\n🧠 Neural Networks Performance Summary:")
        print("-" * 45)

        # Create comparison table
        print(
            f"{'Model':<25} {'Train Acc':<10} {'Test Acc':<10} {'Time (s)':<10} {'Iterations':<12}"
        )
        print("-" * 67)

        for result in nn_models:
            iterations = result.get("iterations", "N/A")
            print(
                f"{result['model_name']:<25} {result['train_accuracy']:<10.3f} "
                f"{result['test_accuracy']:<10.3f} {result['training_time']:<10.3f} {iterations:<12}"
            )

        # Show best neural network
        best_nn = max(nn_models, key=lambda x: x["test_accuracy"])
        print(
            f"\n🏆 Best Neural Network: {best_nn['model_name']} (Test Accuracy: {best_nn['test_accuracy']:.3f})"
        )

        print(f"\n🧠 Neural Network Architecture Analysis:")
        print(f"   • Simple NN (1 layer): Fast baseline, good for simple patterns")
        print(f"   • Deep NN (3 layers): Complex pattern recognition, may overfit")
        print(f"   • Regularized NN: Balanced approach with dropout prevention")

        print(f"\n💡 Key Neural Network Insights:")
        print(f"   • Early stopping prevented overfitting automatically")
        print(f"   • Data already normalized - optimal for neural network training")
        print(f"   • Class balancing improved learning from minority classes")
        print(f"   • Adaptive learning rates helped convergence")

        # Show training efficiency
        total_nn_time = sum(r["training_time"] for r in nn_models)
        print(f"\n⚡ Training Efficiency:")
        print(f"   • Total NN training time: {total_nn_time:.3f} seconds")
        print(
            f"   • Average iterations: {np.mean([r.get('iterations', 0) for r in nn_models]):.1f}"
        )
        print(f"   • All models used early stopping for efficiency")

    print(f"\n Module Neural Network Features:")
    print(f"   ✅ Multiple architectures tested automatically")
    print(f"   ✅ Early stopping and regularization built-in")
    print(f"   ✅ Optimized hyperparameters for time series data")
    print(f"   ✅ Integrated with balanced dataset")
    print(f"   ✅ Automatic feature learning from flattened windows")

else:
    print("❌ No classifier results available.")
    print("💡 Run the comprehensive classification cell to see Neural Network results.")
    print("📋 The module automatically provides:")
    print("   • Simple Neural Network (1 hidden layer)")
    print("   • Deep Neural Network (3 hidden layers)")
    print("   • Regularized Neural Network (with dropout prevention)")
    print("   • Early stopping and validation for all networks")
    print("   • Performance comparison and overfitting analysis")

## 🚀 Implementation and Training Summary (5 minutes)

### Model Performance Comparison
Let's compare all the models we trained and understand their strengths and weaknesses for oil well fault detection.

In [ ]:
# ============================================================
# COMPREHENSIVE ANALYSIS AND TUTORIAL SUMMARY
# ============================================================

print(" Supervised Classification Tutorial Summary")
print("=" * 50)

if "supervised_classifier" in locals() and supervised_classifier is not None:
    print("✅ Comprehensive supervised classification completed successfully!")

    # Show final summary of what was accomplished
    results = supervised_classifier.results

    print(f"\n📊 Tutorial Accomplishments:")
    print(f"   ✅ Loaded windowed data from ALL folds")
    print(f"   ✅ Applied data augmentation for class balancing")
    print(f"   ✅ Used existing data normalization (no additional scaling)")
    print(f"   ✅ Trained {len(results)} different classification models")
    print(f"   ✅ Compared model performance comprehensively")
    print(f"   ✅ Generated performance visualizations")
    print(f"   ✅ Provided practical recommendations")

    # Show algorithm categories covered
    tree_models = [
        r for r in results if "Tree" in r["model_name"] or "Forest" in r["model_name"]
    ]
    svm_models = [r for r in results if "SVM" in r["model_name"]]
    nn_models = [r for r in results if "Neural Network" in r["model_name"]]

    print(f"\n🔬 Algorithms Implemented:")
    print(f"   🌳 Tree-Based: {len(tree_models)} models (Decision Tree, Random Forest)")
    print(f"   ⚡ Support Vector Machines: {len(svm_models)} models (Linear, RBF)")
    print(f"   🧠 Neural Networks: {len(nn_models)} models (Simple, Deep, Regularized)")

    # Show best performers in each category
    if tree_models:
        best_tree = max(tree_models, key=lambda x: x["test_accuracy"])
        print(
            f"\n🏆 Best Tree Model: {best_tree['model_name']} ({best_tree['test_accuracy']:.3f})"
        )

    if svm_models:
        best_svm = max(svm_models, key=lambda x: x["test_accuracy"])
        print(
            f"🏆 Best SVM Model: {best_svm['model_name']} ({best_svm['test_accuracy']:.3f})"
        )

    if nn_models:
        best_nn = max(nn_models, key=lambda x: x["test_accuracy"])
        print(
            f"🏆 Best Neural Network: {best_nn['model_name']} ({best_nn['test_accuracy']:.3f})"
        )

    # Overall best model
    overall_best = max(results, key=lambda x: x["test_accuracy"])
    print(
        f"\n🥇 Overall Best Model: {overall_best['model_name']} ({overall_best['test_accuracy']:.3f})"
    )

    print(f"\n💡 Key Learning Outcomes Achieved:")
    print(f"   📚 Classification Fundamentals: Problem formulation and evaluation")
    print(f"   🌳 Decision Trees & Random Forest: Interpretable ensemble methods")
    print(f"   ⚡ Support Vector Machines: High-dimensional data classification")
    print(f"   🧠 Neural Networks: Deep learning for automatic feature extraction")
    print(f"   📊 Model Comparison: Performance analysis and selection criteria")

    print(f"\n🔧 Technical Implementations:")
    print(
        f"   ✅ Data augmentation for class balancing (using src/data_augmentation.py)"
    )
    print(
        f"   ✅ Comprehensive classification module (src/supervised_classification.py)"
    )
    print(f"   ✅ No redundant normalization (data already preprocessed)")
    print(f"   ✅ Efficient computational strategies (SVM subsampling)")
    print(f"   ✅ Early stopping and regularization for neural networks")

    print(f"\n🎯 Production Readiness:")
    print(f"   • Model code modularized in src/ folder")
    print(f"   • Class balancing pipeline integrated")
    print(f"   • Performance metrics and visualizations available")
    print(f"   • Best model identified with practical recommendations")
    print(f"   • Ready for deployment and further optimization")

    print(f"\n📈 Next Steps:")
    print(f"   1. Implement cross-validation for more robust evaluation")
    print(f"   2. Hyperparameter tuning for the best performing algorithms")
    print(f"   3. Ensemble methods combining multiple top performers")
    print(f"   4. Real-time inference pipeline development")
    print(f"   5. Model monitoring and maintenance procedures")

else:
    print("❌ Comprehensive classification not completed.")
    print("💡 Please run the comprehensive classification cell to:")
    print("   1. Load and prepare windowed data from all folds")
    print("   2. Apply data augmentation for class balancing")
    print("   3. Train all classification algorithms")
    print("   4. Compare model performance")
    print("   5. Generate visualizations and recommendations")

print(f"\n Supervised Classification Tutorial Complete!")
print(f"   You have successfully implemented and compared:")
print(f"   ✅ Decision Trees and Random Forest (8 min)")
print(f"   ✅ Support Vector Machines (4 min)")
print(f"   ✅ Neural Networks and Deep Learning (8 min)")
print(f"   ✅ Implementation and Training Summary (5 min)")
print(f"   ✅ Comprehensive model comparison and analysis")
print(f"   ✅ Production-ready modular implementation")